## Preprocessing

In [1]:
from nilearn import plotting
%matplotlib inline
import os
import json
from nipype.interfaces import fsl 
from nipype.interfaces import spm
from nipype.interfaces.spm import (Realign, SliceTiming, Coregister,  NewSegment,  Normalize12, Smooth)
from nipype.interfaces.spm import Level1Design, EstimateModel, EstimateContrast
from nipype.algorithms.modelgen import SpecifySPMModel
from nipype.interfaces import matlab as mlab
from nipype.interfaces.io import SelectFiles, DataSink
import nipype.interfaces.utility as util 
from nipype.algorithms import rapidart as ra
from nipype.interfaces.utility import Function, IdentityInterface
import nipype.pipeline.engine as pe
import nipype.interfaces.io as nio
from nipype.interfaces.base import Bunch
from nipype import DataGrabber, Workflow, Node
from scipy.io.matlab import loadmat

In [2]:
# necessary to let nipype know about matlab path

In [3]:
spm.SPMCommand.set_mlab_paths(paths=os.path.abspath(os.path.join(os.environ['HOME'], 'Documents/MATLAB/spm12/')), matlab_cmd='/soft/matlab_hd/R2020b/bin/glnxa64/MATLAB -nodesktop -nosplash')

stty: 'standard input': Inappropriate ioctl for device


In [4]:
mlab.MatlabCommand.set_default_matlab_cmd("/soft/matlab_hd/R2020b/bin/glnxa64/MATLAB  -nodesktop -nosplash")
mlab.MatlabCommand.set_default_paths(os.path.abspath(os.path.join(os.environ['HOME'], 'Documents/MATLAB/spm12/')))

In [5]:
# spm.SPMCommand().version

In [6]:
fsl.FSLCommand.set_default_output_type('NIFTI')

In [7]:
base_dir = os.path.join(os.environ['HOME'], 'spmbasics/data/')

In [8]:
experiment_dir = os.path.join(base_dir, 'output')
data_dir = os.path.abspath(os.path.join(base_dir, 'face_rep'))
output_dir = 'nipype'
working_dir = 'workingdir'

# list of subject identifiers
subject_list = ['M03953']

# TR of functional images
TR = 2.


# Smoothing width used during preprocessing
fwhm = [8]

In [9]:
# add how to refer sots

In [10]:
#mat0 = mat = loadmat(os.path.join(data_dir, "sots.mat"), mat_dtype=True, matlab_compatible=True, struct_as_record=True)

In [11]:
mat = loadmat(os.path.join(data_dir, "sots.mat"), mat_dtype=True, matlab_compatible=True, struct_as_record=True, simplify_cells=True)
#sot = mat['sot'][1]

#itemlag = mat['itemlag'][0]
#onsets=[sot[0], sot[1], sot[2], sot[3]],


In [12]:
subjectinfo = [
    Bunch(
        conditions=['N1', 'N2', 'F1', 'F2'],
        onsets=[ mat['sot'][0], mat['sot'][1],  mat['sot'][2],  mat['sot'][3]],
        durations=[[0], [0], [0], [0]],
        amplitudes=None,
        tmod=None,
        pmod=None,
        regressor_names=None,
        regressors=None)
]

In [13]:
# design matrix setting

In [14]:
cond1 = ('positive effect of condition', 'T',
         ['N1', 'N2', 'F1', 'F2'], [1, 1, 1, 1])
cond2 = ('positive effect of condition_dtemo', 'T',
          ['N1', 'N2', 'F1', 'F2'], [1, 1, 1, 1])
cond3 = ('positive effect of condition_ddisp', 'T',
          ['N1', 'N2', 'F1', 'F2'], [1, 1, 1, 1])
# non-famous > famous
fam1 = ('positive effect of Fame', 'T',
        ['N1', 'N2', 'F1', 'F2'], [1, 1, -1, -1])
fam2 = ('positive effect of Fame_dtemp', 'T',
        ['N1', 'N2', 'F1', 'F2'], [1, 1, -1, -1])
fam3 = ('positive effect of Fame_ddisp', 'T',
        ['N1', 'N2', 'F1', 'F2'], [1, 1, -1, -1])
# rep1 > rep2
rep1 = ('positive effect of Rep', 'T',
        ['N1', 'N2', 'F1', 'F2'], [1, -1, 1, -1])
rep2 = ('positive effect of Rep_dtemp', 'T',
        ['N1', 'N2', 'F1', 'F2'], [1, -1, 1, -1])
rep3 = ('positive effect of Rep_ddisp', 'T',
        ['N1', 'N2', 'F1', 'F2'], [1, -1, 1, -1])
int1 = ('positive interaction of Fame x Rep', 'T',
        ['N1', 'N2', 'F1', 'F2'], [-1, -1, -1, 1])
int2 = ('positive interaction of Fame x Rep_dtemp', 'T',
        ['N1', 'N2', 'F1', 'F2'], [1, -1, -1, 1])
int3 = ('positive interaction of Fame x Rep_ddisp', 'T',
        ['N1', 'N2', 'F1', 'F2'], [1, -1, -1, 1])

contf1 = ['average effect condition', 'F', [cond1, cond2, cond3]]
contf2 = ['main effect Fam', 'F', [fam1, fam2, fam3]]
contf3 = ['main effect Rep', 'F', [rep1, rep2, rep3]]
contf4 = ['interaction: Fam x Rep', 'F', [int1, int2, int3]]
contrast_list = [
    cond1, cond2, cond3, fam1, fam2, fam3, rep1, rep2, rep3, int1, int2, int3,
    contf1, contf2, contf3, contf4
]

In [15]:
# SpecifyModel - Generates SPM-specific Model
specifymodel = Node(SpecifySPMModel(concatenate_runs=False,
                                 input_units='scans',
                                 output_units='scans',
                                 time_repetition=TR,
                                 high_pass_filter_cutoff=128,
                                 subject_info = subjectinfo),
                 name="specifymodel")

# Level1Design - Generates an SPM design matrix same as the first level tutorial
eventcategorical = Node(Level1Design(bases={'hrf': {'derivs': [0, 0]}},
                                 timing_units='scans',
                                 interscan_interval=TR,
                                 volterra_expansion_order=1, # no model interction
                                 flags={'mthresh': 0.8},
                                 global_intensity_normalization='none',
                                 microtime_onset=12,
                                 microtime_resolution=24,
                                 #factor_info = [dict(name = 'Fame', levels = 2), # figure out factorized output names
                                 #               dict(name = 'Rep', levels = 2)],
                                 model_serial_correlations='AR(1)'), #matlabbatch{1}.spm.stats.fmri_spec.cvi = 'AR(1)';
                    name="eventcategorical")

# EstimateModel - estimate the parameters of the model
categoricalestimate = Node(EstimateModel(estimation_method={'Classical': 1}),
                      write_residuals=False, 
                      name="categoricalestimate")

# EstimateContrast - estimates contrasts
categoricalconest = Node(EstimateContrast(contrasts = contrast_list),
                    use_derivs=True, 
                    name="categoricalconest")

In [16]:
# Infosource - a function free node to iterate over the list of subject names
infosource = Node(IdentityInterface(fields=['subject_id',
                                            'contrasts'],
                                    contrasts=contrast_list),
                  name="infosource")
infosource.iterables = [('subject_id', subject_list)]


templates = {'func': os.path.join(output_dir, 'event_preproc', '_subject_id_{subject_id}',
                         'swars{subject_id}_0005_*.img'),#sM03953_0005_*.img
             'mc_param': os.path.join(output_dir, 'event_preproc', '_subject_id_{subject_id}',
                         'rp_s{subject_id}_0005_0006.txt')}
selectfiles = Node(SelectFiles(templates,
                               base_directory=experiment_dir,
                               sort_filelist=True),
                   name="selectfiles")

# Datasink - creates output folder for important outputs
datasink = Node(DataSink(base_directory=experiment_dir,
                         container=output_dir),
                name="datasink")

In [17]:
def makelist(item):
    return [item]

In [18]:
# Initiation of the 1st-level analysis workflow
event_cat = Workflow(name='event_cat')
event_cat.base_dir = os.path.join(experiment_dir, working_dir)

# Connect up the 1st-level analysis components
event_cat.connect([(infosource, selectfiles, [('subject_id', 'subject_id')]),
                    (infosource, categoricalconest, [('contrasts', 'contrasts')]),
                    (selectfiles, specifymodel, [('func', makelist),  'functional_runs')]), # [(('smooth.smoothed_files',makelist), 'modelspec.functional_runs')])])

                    (selectfiles, specifymodel, [('mc_param', 'realignment_parameters')]),
                    (specifymodel, eventcategorical, [('session_info','session_info')]),
                    (eventcategorical, categoricalestimate, [('spm_mat_file','spm_mat_file')]),                    
                    (categoricalestimate, categoricalconest, [('spm_mat_file','spm_mat_file'),
                                                    ('beta_images','beta_images'),
                                                   ('residual_image', 'residual_image')]),
                    (categoricalconest, datasink, [('spm_mat_file', 'event_cat.@spm_mat'),
                                              ('spmT_images', 'event_cat.@T'),
                                              ('con_images', 'event_cat.@con'),
                                              ('spmF_images', 'event_cat.@F'),
                                              ('ess_images', 'event_cat.@ess')]),
                    ])

ValueError: too many values to unpack (expected 2)

In [ ]:
event_cat.write_graph(graph2use='colored', format='png', dotfilename='colored_cat.dot', simple_form=True)

In [ ]:
event_cat.write_graph(graph2use='flat', format='png', dotfilename='flat_cat.dot', simple_form=True)

In [ ]:
# Visualize the graph
from IPython.display import Image
Image(filename=os.path.join(event_cat.base_dir, 'event_cat', 'colored_cat.png'))

In [ ]:
event_cat.run('MultiProc', plugin_args={'n_procs': 4})